In [115]:
import numpy as np
from PIL import Image, ImageDraw
from SMM.smm_io.Scan import Scan
from SMM.smm_io.ATF import load_gal

In [35]:
scan = Scan.load_tif('/Volumes/RMW_3/RBD_Panel/SMM-opt/50001918_2020-12-30_S46_A1.tif')[0]
print(scan)

Scan(50001918, 532nm, Standard Green)


In [114]:
gal = load_gal('/Volumes/RMW_3/Compact-Set1.gal', map_blocks=True)
gal.ID
ids = gal.ID.str.extract(r'(\d+)-([A-P])(\d+)\Z').convert_dtypes()
ids.columns = ['Plate', 'Row', 'Column']
sentinels = gal[(ids.Plate=='30') & (ids.Row.str.contains(r'[I-P]'))]
print(sentinels.head())

     BLOCK  COLUMN  ROW     X     Y  DIA   NAME                      ID
0        1       1    1  3300  6000  160  Empty   Dflt-360384-384-30-P1
10       1      11    1  5300  6000  160  Empty   Dflt-360384-384-30-P1
21       1      22    1  7500  6000  160  Empty   Dflt-360384-384-30-L1
241      1      22   11  7500  8000  160  Empty   Dflt-360384-384-30-L1
242      1       1   12  3300  8200  160  Empty  Dflt-360384-384-30-P13


/Users/koehlerchemmodeling/PycharmProjects/SMM-Analysis/SMM/smm_io/ATF.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  return pd_object.str.upper().str.replace('.', '')


In [23]:
Image.fromarray(scan.data[0:6400]).show()

In [84]:
from skimage.filters import *
from skimage.morphology import *
from skimage.feature import *
from skimage.segmentation import *
from skimage.transform import *
from scipy.ndimage import distance_transform_edt

In [128]:
def impose_grid(pil, gal, xpos, ypos, res):
    draw = ImageDraw.Draw(image)
    for _, x, y, r in gal[['X', 'Y', 'DIA']].itertuples():
        x = (x-xpos)//res
        y = (y-ypos)//res
        r = (r/2)//res
        draw.ellipse((x-r, y-r, x+r, y+r), outline='red')

In [95]:
thresh = scan.data[0:6400] > (threshold_local(scan.data[0:6400], 35, method='mean') * 2)
thresh = binary_opening(thresh)
remove_small_objects(thresh, 50, in_place=True)
thresh = distance_transform_edt(~thresh) ** 2
thresh[thresh>100] = 100

In [132]:
image = Image.fromarray(thresh)
impose_grid(image, gal, scan.x_offset, scan.y_offset, scan.resolution)
image.show()